This script is used to merge the TREFHT from different CMIP data   
The results are at "/glade/scratch/zhonghua/CMIP5-pred/TREFHT_CMIP_2006.csv" and "/glade/scratch/zhonghua/CMIP5-pred/TREFHT_CMIP_2061.csv"

In [1]:
import xarray as xr
import datetime
import pandas as pd
import numpy as np
import xesmf as xe
import time
import gc
import matplotlib.pyplot as plt

/glade/work/zhonghua/miniconda3/envs/pangeo/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/glade/work/zhonghua/miniconda3/envs/pangeo/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
CMIP5_ls = ["ACCESS1-0", "ACCESS1-3", "CanESM2", "CNRM-CM5", "CSIRO-Mk3-6-0",
            "GFDL-CM3", "GFDL-ESM2G", "GFDL-ESM2M", "HadGEM2-CC",
            "HadGEM2-ES", "IPSL-CM5A-MR", "MIROC5", "MIROC-ESM", "MIROC-ESM-CHEM",
            "MRI-CGCM3", "MRI-ESM1"]

load_dir = "/glade/scratch/zhonghua/CMIP5-process/"

# 2006

In [3]:
df = pd.read_csv(load_dir + CMIP5_ls[0] + "/2006.csv")
df.head()

,lat,lon,time,QBOT,UBOT,VBOT,TREFHT,FLNS,FSNS,PRECT,PRSN
0,-54.18848,292.5,2006-01-01 12:00:00,0.006171,3.283512,-4.558676,282.584418,-674.473176,199.849947,5.035718e-08,8.046041e-14
1,-54.18848,292.5,2006-01-02 12:00:00,0.006039,8.559795,-3.179946,284.165865,-694.479191,335.075973,4.016366e-08,4.635593e-12
2,-54.18848,292.5,2006-01-03 12:00:00,0.005439,9.976215,-6.571020,282.137580,-681.329872,133.004993,8.498999e-08,5.931727e-12
3,-54.18848,292.5,2006-01-04 12:00:00,0.006600,6.390455,-5.580102,284.307700,-700.360331,259.118935,4.226173e-08,1.097847e-29
4,-54.18848,292.5,2006-01-05 12:00:00,0.005938,5.067210,-1.113135,283.425263,-680.004974,350.880972,7.676413e-08,4.845850e-10


In [4]:
df[CMIP5_ls[0]+"_TREFHT"] = df["TREFHT"]
df_temp=df[["lat","lon","time",CMIP5_ls[0]+"_TREFHT"]]
print(df_temp.shape)
df_temp.head()

(16211228, 4)


,lat,lon,time,ACCESS1-0_TREFHT
0,-54.18848,292.5,2006-01-01 12:00:00,282.584418
1,-54.18848,292.5,2006-01-02 12:00:00,284.165865
2,-54.18848,292.5,2006-01-03 12:00:00,282.137580
3,-54.18848,292.5,2006-01-04 12:00:00,284.307700
4,-54.18848,292.5,2006-01-05 12:00:00,283.425263


In [5]:
for model in CMIP5_ls[1:]:
    print("start to load model", model)
    t0 = time.time()
    df = pd.read_csv(load_dir + model + "/2006.csv")
    print(df.shape)
    df[model+"_TREFHT"] = df["TREFHT"]
    df_new=df[["lat","lon","time",model+"_TREFHT"]]
    print("Load time", time.time()-t0)
    
    print("start to merge model", model)
    t1 = time.time()
    df_temp = pd.merge(df_temp, 
                       df_new,
                       how='left',
                       on=['lat', 'lon', 'time'])
    print(df_temp.shape)
    print("Merge time", time.time()-t1)
    print("\n")
    del df, df_new
    gc.collect()

start to load model ACCESS1-3
(16211228, 11)
Load time 32.976898193359375
start to merge model ACCESS1-3
(16211228, 5)
Merge time 13.123344659805298


start to load model CanESM2
(16202350, 11)
Load time 32.47905707359314
start to merge model CanESM2
(16211228, 6)
Merge time 12.971054315567017


start to load model CNRM-CM5
(16211228, 11)
Load time 33.320263147354126
start to merge model CNRM-CM5
(16211228, 7)
Merge time 13.141857862472534


start to load model CSIRO-Mk3-6-0
(16202350, 11)
Load time 34.066142082214355
start to merge model CSIRO-Mk3-6-0
(16211228, 8)
Merge time 13.36143183708191


start to load model GFDL-CM3
(16202350, 11)
Load time 34.040241718292236
start to merge model GFDL-CM3
(16211228, 9)
Merge time 13.375513792037964


start to load model GFDL-ESM2G
(16202350, 11)
Load time 33.094988107681274
start to merge model GFDL-ESM2G
(16211228, 10)
Merge time 13.486825227737427


start to load model GFDL-ESM2M
(16202350, 11)
Load time 32.98670792579651
start to merge mode

In [6]:
df_temp.head()

,lat,lon,time,ACCESS1-0_TREFHT,ACCESS1-3_TREFHT,CanESM2_TREFHT,CNRM-CM5_TREFHT,CSIRO-Mk3-6-0_TREFHT,GFDL-CM3_TREFHT,GFDL-ESM2G_TREFHT,GFDL-ESM2M_TREFHT,HadGEM2-CC_TREFHT,HadGEM2-ES_TREFHT,IPSL-CM5A-MR_TREFHT,MIROC5_TREFHT,MIROC-ESM_TREFHT,MIROC-ESM-CHEM_TREFHT,MRI-CGCM3_TREFHT,MRI-ESM1_TREFHT
0,-54.18848,292.5,2006-01-01 12:00:00,282.584418,283.234302,280.941966,282.729592,283.485401,280.837664,283.669874,284.408093,282.946906,282.709878,284.288727,281.730792,281.703448,282.858946,281.095663,283.910928
1,-54.18848,292.5,2006-01-02 12:00:00,284.165865,282.261594,281.471671,282.935227,282.192655,280.672503,285.101322,284.773204,282.608810,284.707193,281.467266,282.247388,281.403317,283.118593,283.249040,282.167630
2,-54.18848,292.5,2006-01-03 12:00:00,282.137580,282.305401,280.834231,281.571139,282.745297,282.998008,284.617067,283.688546,282.730244,282.265517,278.792592,283.895594,281.758941,282.675634,284.045677,287.865810
3,-54.18848,292.5,2006-01-04 12:00:00,284.307700,281.926324,280.346198,280.800636,284.165688,283.238552,283.336118,284.650378,283.553190,283.404724,278.258979,284.584368,282.159510,283.898273,281.874078,286.058953
4,-54.18848,292.5,2006-01-05 12:00:00,283.425263,284.439676,280.899456,281.791321,284.446995,284.235573,285.058910,284.510261,281.552386,284.126804,279.911558,284.960310,282.108025,283.005493,281.422140,284.859729


In [7]:
t0 = time.time()
df_temp.to_csv("/glade/scratch/zhonghua/CMIP5-pred/TREFHT_CMIP_2006.csv",index=False)
print(time.time()-t0)

344.33048582077026


# 2061

In [8]:
df = pd.read_csv(load_dir + CMIP5_ls[0] + "/2061.csv")
df.head()

,lat,lon,time,QBOT,UBOT,VBOT,TREFHT,FLNS,FSNS,PRECT,PRSN
0,-54.18848,292.5,2061-01-01 12:00:00,0.005191,1.630520,7.447552,282.352807,-651.230319,360.661928,9.379762e-09,1.383378e-13
1,-54.18848,292.5,2061-01-02 12:00:00,0.005009,4.094651,6.619579,282.008058,-643.839537,389.004302,3.864960e-10,-5.190624e-29
2,-54.18848,292.5,2061-01-03 12:00:00,0.006009,4.246638,-0.811103,282.127323,-672.217209,105.271386,6.586176e-08,1.192860e-29
3,-54.18848,292.5,2061-01-04 12:00:00,0.005814,8.742697,-2.595085,282.093791,-679.514150,167.833096,3.216877e-08,1.273468e-27
4,-54.18848,292.5,2061-01-05 12:00:00,0.005536,6.887794,3.380637,282.490602,-675.776350,325.702779,1.989858e-08,4.911568e-11


In [9]:
df[CMIP5_ls[0]+"_TREFHT"] = df["TREFHT"]
df_temp=df[["lat","lon","time",CMIP5_ls[0]+"_TREFHT"]]
print(df_temp.shape)
df_temp.head()

(16211228, 4)


,lat,lon,time,ACCESS1-0_TREFHT
0,-54.18848,292.5,2061-01-01 12:00:00,282.352807
1,-54.18848,292.5,2061-01-02 12:00:00,282.008058
2,-54.18848,292.5,2061-01-03 12:00:00,282.127323
3,-54.18848,292.5,2061-01-04 12:00:00,282.093791
4,-54.18848,292.5,2061-01-05 12:00:00,282.490602


In [10]:
for model in CMIP5_ls[1:]:
    print("start to load model", model)
    t0 = time.time()
    df = pd.read_csv(load_dir + model + "/2061.csv")
    print(df.shape)
    df[model+"_TREFHT"] = df["TREFHT"]
    df_new=df[["lat","lon","time",model+"_TREFHT"]]
    print("Load time", time.time()-t0)
    
    print("start to merge model", model)
    t1 = time.time()
    df_temp = pd.merge(df_temp, 
                       df_new,
                       how='left',
                       on=['lat', 'lon', 'time'])
    print(df_temp.shape)
    print("Merge time", time.time()-t1)
    print("\n")
    del df, df_new
    gc.collect()
df_temp.head()

start to load model ACCESS1-3
(16211228, 11)
Load time 32.56583070755005
start to merge model ACCESS1-3
(16211228, 5)
Merge time 13.07293701171875


start to load model CanESM2
(16202350, 11)
Load time 32.580668210983276
start to merge model CanESM2
(16211228, 6)
Merge time 13.427116632461548


start to load model CNRM-CM5
(16211228, 11)
Load time 34.117717266082764
start to merge model CNRM-CM5
(16211228, 7)
Merge time 13.433972597122192


start to load model CSIRO-Mk3-6-0
(16202350, 11)
Load time 34.02495455741882
start to merge model CSIRO-Mk3-6-0
(16211228, 8)
Merge time 13.544740676879883


start to load model GFDL-CM3
(16202350, 11)
Load time 33.918466567993164
start to merge model GFDL-CM3
(16211228, 9)
Merge time 13.641599416732788


start to load model GFDL-ESM2G
(16202350, 11)
Load time 46.598592042922974
start to merge model GFDL-ESM2G
(16211228, 10)
Merge time 21.60299777984619


start to load model GFDL-ESM2M
(16202350, 11)
Load time 38.62099742889404
start to merge model 

,lat,lon,time,ACCESS1-0_TREFHT,ACCESS1-3_TREFHT,CanESM2_TREFHT,CNRM-CM5_TREFHT,CSIRO-Mk3-6-0_TREFHT,GFDL-CM3_TREFHT,GFDL-ESM2G_TREFHT,GFDL-ESM2M_TREFHT,HadGEM2-CC_TREFHT,HadGEM2-ES_TREFHT,IPSL-CM5A-MR_TREFHT,MIROC5_TREFHT,MIROC-ESM_TREFHT,MIROC-ESM-CHEM_TREFHT,MRI-CGCM3_TREFHT,MRI-ESM1_TREFHT
0,-54.18848,292.5,2061-01-01 12:00:00,282.352807,285.530828,282.494501,282.720654,286.479559,278.514279,283.859071,283.892807,285.496719,285.289276,279.826908,286.955642,284.584468,285.392617,281.193364,288.080281
1,-54.18848,292.5,2061-01-02 12:00:00,282.008058,285.491256,281.740900,284.504972,285.999094,279.099595,281.983461,284.106235,285.342134,282.361808,279.963400,284.691207,283.324617,284.914037,280.909569,287.522401
2,-54.18848,292.5,2061-01-03 12:00:00,282.127323,283.639416,282.001485,283.201536,285.350658,279.403592,283.058618,284.840654,284.490271,283.060698,279.896029,285.003618,281.604435,283.819696,280.561885,284.922727
3,-54.18848,292.5,2061-01-04 12:00:00,282.093791,285.175198,282.894330,282.714644,285.670553,280.488014,283.724541,286.779876,283.558529,285.895106,279.539351,285.092182,282.095465,284.227813,281.142769,282.924857
4,-54.18848,292.5,2061-01-05 12:00:00,282.490602,284.795930,282.779741,282.579663,284.095221,283.813011,283.702484,286.126766,284.685841,285.033841,279.176430,285.165195,282.226317,283.808029,282.248676,286.006486


In [11]:
t0 = time.time()
df_temp.to_csv("/glade/scratch/zhonghua/CMIP5-pred/TREFHT_CMIP_2061.csv",index=False)
print(time.time()-t0)

365.9421126842499
